# Intro


Lately, I've been following [FastHTML](https://about.fastht.ml/) from a distance. As someone who sticks to backend Python development, frontend development has always been a bit foreign to me, but I'm interested in giving it a shot. FastHTML feels like a good way to get started with some basics by building small apps.

I've also noticed a lot of chatter on X about [Colpali](https://github.com/illuin-tech/colpali) and document retrieval with vision language models, which caught my attention. I like exploring new stuff so I want to see what that is all about.

On top of that, I'm still enjoying [Modal](https://modal.com/), which I’ve written about before [here](https://drchrislevy.github.io/posts/modal_fun/modal_blog.html) and [here](https://drchrislevy.github.io/posts/intro_modal/intro_modal.html). I thought it would be fun to combine these tools into a simple app and see what I can learn from it.

All the code for this project is in this [folder](https://github.com/DrChrisLevy/DrChrisLevy.github.io/blob/main/posts/colpali/README.md).
The main code is the following:

- [multi_modal_rag.py](https://github.com/DrChrisLevy/DrChrisLevy.github.io/blob/main/posts/colpali/multi_modal_rag.py) - A Modal app running on CPU that runs the multimodal retrieval logic.
- [pdf_retriever.py](https://github.com/DrChrisLevy/DrChrisLevy.github.io/blob/main/posts/colpali/pdf_retriever.py) - A Modal app running on GPU which processes and caches images/embeddings for each PDF and runs inference for ColPali.
- [utils.py](https://github.com/DrChrisLevy/DrChrisLevy.github.io/blob/main/posts/colpali/utils.py) - some simple utility functions for logging and generating unique folder names in the Modal Volumes.
- [main.py](https://github.com/DrChrisLevy/DrChrisLevy.github.io/blob/main/posts/colpali/main.py) - the FastHTML app that runs the frontend.
- [colpali_blog.ipynb](https://github.com/DrChrisLevy/DrChrisLevy.github.io/blob/main/posts/colpali/colpali_blog.ipynb) - a notebook that I used to generate the blog post for this project.

See the [README](https://github.com/DrChrisLevy/DrChrisLevy.github.io/blob/main/posts/colpali/README.md) for more details.

# ColPali

There are already so many great resources out there about ColPali. Checkout the resources [below](#resources) for more information.
I will give a quick overview.

I have already deployed ColPali to Modal as a remote function I can call, running on an A10 GPU.

```
modal deploy pdf_retriever.py
```

Remember that with Modal, you only pay for compute when running requests in active containers. My deployed app
can sit there idle without costing me anything!

![ColPali Model Deployed on Modal](imgs/colpali_modal.png)


There are a couple functions I have decorated with `@modal.method()` within the `PDFRetriever` class:

-  `def forward(self, inputs)` --> [here](https://github.com/DrChrisLevy/DrChrisLevy.github.io/blob/main/posts/colpali/pdf_retriever.py#L72)
-  `def top_pages(self, pdf_url, queries, use_cache=True, top_k=1)` --> [here](https://github.com/DrChrisLevy/DrChrisLevy.github.io/blob/main/posts/colpali/pdf_retriever.py#L103)


Let's look at the `forward` function first as it can be used to run inference on a list of strings or images to get the embeddings.

First we will pass in text inputs to ColPali.

In [1]:
# | warning: false
import modal

forward = modal.Function.lookup("pdf-retriever", "PDFRetriever.forward")
embeddings_batch = forward.remote(["How does the latency between ColPali and standard retrieval methods compare?"])
assert len(embeddings_batch) == 1  # we passed in one document i.e. batch size of 1
embeddings = embeddings_batch[0]
print(embeddings.shape)
embeddings

/Users/christopher/personal_projects/DrChrisLevy.github.io/posts/colpali/env/lib/python3.11/site-packages/torch/_subclasses/functional_tensor.py:258: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


torch.Size([28, 128])


tensor([[ 0.1572, -0.0240,  0.0942,  ..., -0.0278, -0.0791, -0.0129],
        [-0.0688, -0.1260,  0.0038,  ..., -0.0073, -0.1162,  0.0962],
        [ 0.0413, -0.1055, -0.1055,  ..., -0.0055, -0.2178,  0.1406],
        ...,
        [-0.0825, -0.0444, -0.0674,  ..., -0.0327, -0.1504,  0.1670],
        [ 0.1465,  0.0016, -0.1338,  ...,  0.0127, -0.2119,  0.1191],
        [ 0.1641, -0.0405, -0.1338,  ...,  0.0175, -0.2080,  0.1177]],
       dtype=torch.bfloat16)

The first thing to note is that we don't get a single dense embedding vector.
Traditionally that is the case where a single vector is used to represent one input.
But ColPali is generating ColBERT-style multi-vector representations of the input.
With the late interaction paradigm you get back multiple embeddings, one per input **token**.
Each embedding is 128-dimensional. 


ColPali is trained to take image documents as input.
It was trained on query-document pairs where each document is a page of a PDF.
Each PDF page ("document") is treated as an image. It uses a vision language model to create 
multi-vector embeddings purely from visual document features.

Consider the following image of a PDF page from the ColPali paper:

![ColPali Paper PDF Page 2](imgs/colpali_paper_page_sample.png)
We can pass this image to the `forward` function and get the embeddings back.
The ColPali model divides each page image into a 32 x 32 = 1024 patches.
In addition to the image grid patches, ColPali includes 6 instruction text tokens that are prepended to the image input. 
These tokens represent the text: "Describe the image." Combining the image grid patches and the instruction tokens, we get:
1024 (image patches) + 6 (instruction tokens) = 1030 total patches/embeddings.




In [2]:
from PIL import Image

img = Image.open("imgs/colpali_paper_page_sample.png")
embeddings = forward.remote([img])[0]
print(embeddings.shape)
embeddings

torch.Size([1030, 128])


tensor([[-0.1562, -0.0396, -0.0908,  ...,  0.1426, -0.1113,  0.1079],
        [-0.1260,  0.0427,  0.0991,  ..., -0.0286, -0.0170,  0.0786],
        [-0.1621,  0.0297,  0.0874,  ..., -0.0255, -0.0168,  0.0625],
        ...,
        [ 0.1045, -0.0178,  0.0522,  ..., -0.0986, -0.1011, -0.0366],
        [ 0.0078,  0.0674,  0.0674,  ..., -0.0226, -0.0479, -0.0908],
        [ 0.0062,  0.0623,  0.1396,  ...,  0.0264, -0.1699, -0.1533]],
       dtype=torch.bfloat16)

Using the ColPali model we produce multi-vector embeddings per page which can be indexed.
At query time, we use the same model to generate multi-vector embeddings for the query. 
So both queries and documents are represented as sets of vectors rather than single vector.

The MaxSim (Maximum Similarity) scoring function is used to compute the similarity between query embeddings and document embeddings.
The scoring function performs the following steps:

- Computes dot products between all query token embeddings and all document page patch embeddings
- Applies a max reduce operation over the patch dimension
- Performs a sum reduce operation over the query tokens

There is a great and simple explanation in this [blog post](Both queries and documents are represented as sets of vectors rather than single vector.)

I have wrapped the logic for a given PDF url and query/question within the deployed Modal function 

`def top_pages(self, pdf_url, queries, use_cache=True, top_k=1)`.

The function takes a `pdf_url` and a list of `queries` (questions) and returns the top `top_k` pages for each query/question.
The use of ColPali and the MaxSim scoring function allows us to retrieve the most relevant pages from the PDF
that will assist in answering the question

In [3]:
get_top_pages = modal.Function.lookup("pdf-retriever", "PDFRetriever.top_pages")
pdf_url = "https://arxiv.org/pdf/2407.01449"
top_pages = get_top_pages.remote(pdf_url, queries=["How does the latency between ColPali and standard retrieval methods compare?"], top_k=3)[0]
top_pages

[1, 0, 4]

This first returned index page `1` is actually the second page of the PDF since we start counting from `0`.
And that page being returned is the image we saw earlier from the ColPali paper. It's really cool
because the answer is found in the figure on that page.

# Generating the Answer

Once we have the top pages/images as context, we can pass them along with the query/question to a vision language model to generate an answer.
The images are passed as the context and the question/query is passed as text. I have this logic deployed in a Modal Application as well
running on CPU. It communicates with the other deployed ColPali Modal app running on the GPU when it needs to compute the embeddings.
I am using OpenAI's `gpt-4o-mini` for the vision language model to generate the answer with the provided image context and question.

```
modal deploy multi_modal_rag.py
```

The deployed Modal function [here](https://github.com/DrChrisLevy/DrChrisLevy.github.io/blob/main/posts/colpali/multi_modal_rag.py#L62) is 

```
def answer_question_with_image_context(pdf_url, query, top_k=1, use_cache=True, max_new_tokens=2000, additional_instructions=""):
```



In [4]:
answer_question_with_image_context = modal.Function.lookup("multi-modal-rag", "answer_question_with_image_context")
res = answer_question_with_image_context.remote_gen(
    pdf_url="https://arxiv.org/pdf/2407.01449", query="How does the latency between ColPali and standard retrieval methods compare?", top_k=5
)
answer = "".join([chunk for chunk in res if type(chunk) == str])
print(answer)

The latency comparison between ColPali and standard retrieval methods indicates that ColPali is significantly faster. Specifically:

- **ColPali**: 0.39 seconds per page.
- **Standard Retrieval**: 7.22 seconds per page.

This demonstrates that ColPali achieves better performance in terms of latency while maintaining a stronger relevance score in document retrieval tasks.


# FastHTML App

To demo the FastHTML App I created, I will share images and videos of running it locally.
The entire app is in the code [main.py](https://github.com/DrChrisLevy/DrChrisLevy.github.io/blob/main/posts/colpali/main.py).

```
python main.py
```

Here is what the app looks like when you first load it up:

![](imgs/fasthtml_demo1.png)


Here are two videos of running the app and asking questions about the ColPali paper.


{{< video https://www.youtube.com/watch?v=YoXkFCA0qC8 >}}


{{< video https://www.youtube.com/watch?v=AR7h95IppMU >}}


This PDF url of the ColPali paper was already processed and cached which means I already stored the embeddings and images
inside volumes on Modal. So it loads the document embeddings and images very quickly. Also, the Modal container was warm
and running so there were no cold start delays.

In this next video I will demo the app with a new PDF url that was not processed and cached yet.
I will also send the requests to the backend when the Modal containers are idle.
These requests will trigger the Modal containers to start up and run the inference.
It will take longer but you will see how everything is logged from the backend in the terminal window I created.
It uses server-sent events (SSE) to stream the logs to the frontend so you can see what is happening in the backend.
This example will use a longer PDF from Meta, [Movie Gen: A Cast of Media Foundation Models](https://ai.meta.com/static-resource/movie-gen-research-paper),
which is 92 pages.

{{< video https://www.youtube.com/watch?v=Eu6QJjD73N0&list=PLSF4aA8KYOVwRv4I6hBgdlfw4uy5JLE0V&index=3 >}}

This next video runs the same PDF and question a second time. Now that all the images and document embeddings are cached
in a volume on Modal, everything is much faster. This is also using a warm Modal container so there were no cold start delays.
Most of the time is spent in the OpenAI API call which takes five images as input and streams back the text response.

{{< video https://www.youtube.com/watch?v=Z-EOqVBibSY&list=PLSF4aA8KYOVwRv4I6hBgdlfw4uy5JLE0V&index=4 >}}

# Highlights

There are a few highlights I want to call out.
The first is the use of server-sent events (SSE) to stream the logs to the frontend.
The backend code is running in the cloud on Modal's infrastructure.
In the frontend code I created the terminal looking window with this [code](https://github.com/DrChrisLevy/DrChrisLevy.github.io/blob/main/posts/colpali/main.py#L54-L60).
It continually calls the `/poll-queue` endpoint to get the latest logs from Modal and streams them via SSE. 
In Modal I am using a Queue to collect the logs. Throughout my Modal application code I use these [functions](https://github.com/DrChrisLevy/DrChrisLevy.github.io/blob/main/posts/colpali/utils.py#L18-L29). Anytime I want to log a message I just call `log_to_queue`. It gets placed on the queue and then
`read_from_queue` is used to pop the message off the queue and display it. It's a fun and neat way to provide more visibility
to the frontend about what the backend is doing.

![](imgs/fasthtml_demo2.png)


Another highlight is the use of Modal's volume functionality. 
I use a volume to store the images and document embeddings for each PDF that is processed.
This way if the PDF is used a second time, the images and embeddings are stored to 
the Volume for fast retrieval. This avoids having to call ColPali processing and PDF
processing for each question/query related to the same document.

One final highlight was streaming the OpenAI response back to the frontend in markdown format via SSE.
This took me a while to figure out how to do. On the frontend I did [this](https://github.com/DrChrisLevy/DrChrisLevy.github.io/blob/main/posts/colpali/main.py#L131).
There could be better ways to do this but it works for now. Big shout out to `@Frax` and `@Phi` from the [FastHTML Discord channel](https://discord.com/channels/689892369998676007/1296050761414742127) for helping me out with that. Streaming from Modal was really easy. I just made used of `yield` [here](https://github.com/DrChrisLevy/DrChrisLevy.github.io/blob/main/posts/colpali/multi_modal_rag.py#L78-82) and `remote_gen` [here](https://github.com/DrChrisLevy/DrChrisLevy.github.io/blob/main/posts/colpali/main.py#L84).

![](imgs/modal_volumes1.png)

There is a folder for each PDF processed (for images and embeddings).

![](imgs/modal_volumes2.png)

![](imgs/modal_volumes3.png)

Each image for each page is stored in the volume like this:
![](imgs/modal_volumes4.png)

And all the document embeddings are stored in Pickle format in a file called `embeddings.pkl`.
![](imgs/modal_volumes5.png)

Since I am only allowing to ask questions about a single PDF at a time, there is no need for fancy vector DBs etc.
The embeddings for a specific PDF are cached and can be loaded into memory very quickly when needed.
When a new PDF comes along that is not cached, we process it, and then store the images and embeddings in the volume.
You can see all the details about PDF processing and ColPali inference in the [PDFRetriever class](https://github.com/DrChrisLevy/DrChrisLevy.github.io/blob/main/posts/colpali/pdf_retriever.py).



# Resources

In no particular order:

- [Colpali paper](https://arxiv.org/pdf/2407.01449v2)
- [Colbert paper](https://arxiv.org/pdf/2004.12832)
- [Colbert V2 paper](https://arxiv.org/pdf/2112.01488)
- [PaliGemma](https://arxiv.org/pdf/2407.07726)
- [A little pooling goes a long way for multi-vector representations: Blog answer.ai](https://www.answer.ai/posts/colbert-pooling.html)
    - [Reducing the Footprint of Multi-Vector Retrieval with Minimal Performance Impact via Token Pooling, Paper](https://arxiv.org/pdf/2409.14683)
- [PLAID paper](https://arxiv.org/pdf/2205.09707)
- [Beyond the Basics of Retrieval for Augmenting Generation (w/ Ben Clavié), Youtube Talk](https://www.youtube.com/watch?v=0nA5QG3087g)
- [RAG is more than dense embedding, Google Slides, Ben Clavié](https://docs.google.com/presentation/d/1Zczs5Sk3FsCO06ZLDznqkOOhbTe96PwJa4_7FwyMBrA/edit#slide=id.p)
- The quick start in the README [Original ColPali Repo](https://github.com/illuin-tech/colpali) as well as the sample [inference code](https://github.com/illuin-tech/colpali/blob/main/scripts/infer/run_inference_with_python.py)
- [Hugging Face Model Cards](https://huggingface.co/vidore/colpali-v1.2)
- [The Future of Search: Vision Models and the Rise of Multi-Model Retrieval](https://mcplusa.com/the-future-of-search-vision-models-and-the-rise-of-multi-model-retrieval/)
- [Scaling ColPali to billions of PDFs with Vespa](https://blog.vespa.ai/scaling-colpali-to-billions/)
- [Beyond Text: The Rise of Vision-Driven Document Retrieval for RAG](https://blog.vespa.ai/the-rise-of-vision-driven-document-retrieval-for-rag/)
- [Vision Language Models Explained](https://huggingface.co/blog/vlms)
- [Document Similarity Search with ColPali](https://huggingface.co/blog/fsommers/document-similarity-colpali)
- [Jo Kristian Bergum: X](https://x.com/jobergum)
- [Manuel Faysse: X](https://x.com/ManuelFaysse)
- [Tony Wu: X](https://x.com/tonywu_71)
- [Omar Khattab: X](https://x.com/lateinteraction?lang=en)
- [fastHTML](https://about.fastht.ml/)